In [14]:
import pandas as pd

In [15]:
df = pd.read_csv('dataset.csv')

In [16]:
df.head()

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...


In [17]:
df = df[['Title','Plot','Genre']]

In [18]:
df.head()

,Title,Plot,Genre
0,Kansas Saloon Smashers,"A bartender is working at a saloon, serving dr...",unknown
1,Love by the Light of the Moon,"The moon, painted with a smiling face hangs ov...",unknown
2,The Martyred Presidents,"The film, just over a minute long, is composed...",unknown
3,"Terrible Teddy, the Grizzly King",Lasting just 61 seconds and consisting of two ...,unknown
4,Jack and the Beanstalk,The earliest known adaptation of the classic f...,unknown


In [19]:
genres = df.Genre.unique()
print(genres)
genres.size

['unknown' 'western' 'comedy' ... 'adventure, romance, fantasy film' 'ero'
 'horror romantic comedy']


2265

In [20]:
df = df[df.Genre!='unknown']
df = df[~df.Genre.str.contains(r",|/| |-", regex=True)]
df.head()

,Title,Plot,Genre
6,The Great Train Robbery,The film opens with two bandits breaking into ...,western
7,The Suburbanite,The film is about a family who move to the sub...,comedy
10,Dream of a Rarebit Fiend,The Rarebit Fiend gorges on Welsh rarebit at a...,short
13,Daniel Boone,Boone's daughter befriends an Indian maiden as...,biographical
14,How Brown Saw the Baseball Game,Before heading out to a baseball game at a nea...,comedy


In [21]:
df.count

<bound method DataFrame.count of                                  Title  \
6              The Great Train Robbery   
7                      The Suburbanite   
10            Dream of a Rarebit Fiend   
13                        Daniel Boone   
14     How Brown Saw the Baseball Game   
...                                ...   
34875     Once Upon a Time in Anatolia   
34876                            Selam   
34882               Çalgı Çengi İkimiz   
34883                     Olanlar Oldu   
34885               İstanbul Kırmızısı   

                                                    Plot         Genre  
6      The film opens with two bandits breaking into ...       western  
7      The film is about a family who move to the sub...        comedy  
10     The Rarebit Fiend gorges on Welsh rarebit at a...         short  
13     Boone's daughter befriends an Indian maiden as...  biographical  
14     Before heading out to a baseball game at a nea...        comedy  
...                     

In [27]:
df.Genre.value_counts().head(10)

Genre
drama        5964
comedy       4379
horror       1167
action       1098
thriller      966
romance       923
western       865
crime         568
adventure     526
musical       467
Name: count, dtype: int64

In [36]:
restrictions = ['drama', 'comedy', 'horror', 'action', 'thriller', 'romance']

df = df[df['Genre'].isin(restrictions)]
df

,Title,Plot,Genre
7,The Suburbanite,The film is about a family who move to the sub...,comedy
14,How Brown Saw the Baseball Game,Before heading out to a baseball game at a nea...,comedy
15,Laughing Gas,The plot is that of a black woman going to the...,comedy
16,The Adventures of Dollie,On a beautiful summer day a father and mother ...,drama
17,The Black Viper,A thug accosts a girl as she leaves her workpl...,drama
...,...,...,...
34874,White as Snow,Hasan is a twelve-year-old boy living with his...,drama
34875,Once Upon a Time in Anatolia,"Through the night, three cars carry a small gr...",drama
34876,Selam,The film opens with a Senegalese boy named Kha...,drama
34882,Çalgı Çengi İkimiz,"Two musicians, Salih and Gürkan, described the...",comedy
